In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import fiftyone as fo

from PIL import Image
from torchvision import transforms

from datetime import datetime
import random
import torch
import lightning as L
from tqdm import tqdm

from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import LearningRateFinder

from lightning.pytorch.utilities.model_summary import ModelSummary
from lightning.pytorch.tuner import Tuner

from torch.utils.data import DataLoader

from model.model_light import FishSeg
from model.dataset import SimpleFishialFishDataset
from model.utils import *
from argparse import ArgumentParser

from model.dataset import FishialSegmentDatasetFoOnlineCuting


In [2]:
MODEL_PATH = '/home/andrew/Andrew/MMLAB/FishialSEGM/FPN_resnet18_416_2024_07_30_10_54/checkpoints/sample-epoch=82-valid_img_iou=0.95345.ckpt'


encoder_type = "FPN"
backbone = "resnet18"
in_channels = 3
out_classes = 1

IMAGE_SIZE = 416

In [ ]:
fo_dataset = fo.load_dataset('SEGM-2024-V0.8')

train_view = fo_dataset.match(~fo.ViewField("tags").contains('val'))
val_view = fo_dataset.match_tags("val")

train_dataset = FishialSegmentDatasetFoOnlineCuting(train_view, aug = False, image_size = IMAGE_SIZE)
valid_dataset = FishialSegmentDatasetFoOnlineCuting(val_view, aug = False, image_size = IMAGE_SIZE)

In [ ]:
model = FishSeg(encoder_type, backbone, in_channels=3, out_classes=1, load_checkpoint = MODEL_PATH)
model.eval()
model.cpu()

In [ ]:
iumage = train_dataset[0]['src_img']
plt.imshow(iumage)
plt.show()

In [ ]:
data_pred = np.array(data_pred)
np.save('data_iou_idx.npy', data_pred)

In [ ]:
count_to_visualize = 10
indices = random.sample(range(0,len(train_dataset)), count_to_visualize)
sample_id = 21163

data_pred = [[],[]]
for sample_id in tqdm(range(len(train_dataset))):
    
    sample = train_dataset[sample_id]
    
    gt_mask = sample['mask'][0]
    
    height, width, _ = sample['src_img'].shape
    
    x_tensor = sample['image'].unsqueeze(0)
    
    with torch.no_grad():
        logits = model(x_tensor)
        
    pr_mask = logits.sigmoid()[0][0].numpy()
    
#     pr_mask = resize_logits_mask_pil(pr_mask, width, height)
#     gt_mask = resize_logits_mask_pil(gt_mask, width, height)

    intersection = np.logical_and(pr_mask, gt_mask)
    union = np.logical_or(pr_mask, gt_mask)
    iou = np.sum(intersection) / np.sum(union)
    
    if iou < 0.5:
        print(iou, sample_id)
    
    data_pred[0].append(iou)
    data_pred[1].append(sample_id)

#     print(f"IoU: {iou}")

#     visualize(
#         image=sample['src_img'], 
#         ground_truth_mask=gt_mask, 
#         predicted=pr_mask
#     )

In [ ]:
croped_model = model.model
croped_model.eval()
croped_model.cpu()

example_forward_input = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE)

# Trace a specific method and construct `ScriptModule` with
# a single `forward` method
module = torch.jit.trace(croped_model.forward, example_forward_input)

In [ ]:
import torch.onnx

# Define the path to save the ONNX model
onnx_model_path = f"saved_models/segmentator/model_resnet18_{IMAGE_SIZE}.onnx"

# Load a pre-trained model (in this case, ResNet18)
# model = models.resnet18(pretrained=True)
# model.eval()  # Set the model to evaluation mode

# Create an input tensor with the corresponding shape (batch_size, channels, height, width)
dummy_input = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE)

# Export the model to ONNX format
torch.onnx.export(croped_model,              # The model to export
                  dummy_input,               # The input tensor
                  onnx_model_path,           # The path to save the model
                  export_params=True,        # Export the parameters as well
                  opset_version=11,          # The ONNX version
                  do_constant_folding=False, # Disable constant folding optimization
                  input_names=['input'],     # The names of the input layers
                  output_names=['output'],   # The names of the output layers
                  dynamic_axes={'input': {0: 'batch_size'},   # Dynamic batch size
                                'output': {0: 'batch_size'}})

print(f"Model successfully exported to {onnx_model_path}")

In [ ]:
SAVE_PATH = f"saved_models/segmentator/model_resnet18_{IMAGE_SIZE}.ts"
module.save(SAVE_PATH)

In [ ]:
new_model = torch.jit.load(SAVE_PATH)

In [ ]:
import onnxruntime
ort_session = onnxruntime.InferenceSession(onnx_model_path)

In [ ]:
batch_size = 1
x = torch.randn(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE, requires_grad=False)

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: x.numpy()}
ort_outs = ort_session.run(None, ort_inputs)

# with torch.no_grad():
torch_out = croped_model(x)

np_onnx_oputput = np.array(ort_outs)
full_model_output = torch_out.detach().numpy()

print(np.sum(np_onnx_oputput - full_model_output))
# # compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(full_model_output, np_onnx_oputput, rtol=1e-03, atol=1e-05)

In [ ]:
# model.eval()
new_model.eval()

In [ ]:
loader = transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])


In [ ]:
x_tensor

In [ ]:
count_to_visualize = 10
view = data.take(count_to_visualize)

for sample_id, sample in enumerate(view):
    
    filepath = sample.filepath
    polygon = sample.polyline.points[0]
    
    pil_image = Image.open(filepath)
    width, height = pil_image.size
    
    gt_mask = create_mask(polygon, height, width, color = (100,123,234))
    
    x_tensor = loader(pil_image).unsqueeze(0)
    
    with torch.no_grad():
        logits = new_model(x_tensor)
        logits_std = model(x_tensor)
        logits_croped = croped_model(x_tensor)

        ort_inputs = {ort_session.get_inputs()[0].name: x_tensor.numpy()}
        ort_outs = torch.tensor(ort_session.run(None, ort_inputs)[0])
        
    pr_mask_onnx = ort_outs.sigmoid()[0][0].numpy()
    pr_mask_onnx = resize_logits_mask_pil(pr_mask_onnx, width, height)
    
    pr_mask_croped = logits_croped.sigmoid()[0][0].numpy()
    pr_mask_croped = resize_logits_mask_pil(pr_mask_croped, width, height)
    
    pr_mask_std = logits_std.sigmoid()[0][0].numpy()
    pr_mask_std = resize_logits_mask_pil(pr_mask_std, width, height)
    
    pr_mask = logits.sigmoid()[0][0].numpy()
    pr_mask = resize_logits_mask_pil(pr_mask, width, height)
    
    visualize(
        image=pil_image, 
        ground_truth_mask=gt_mask,
        predicted_full=pr_mask_std,
        pr_mask_croped = pr_mask_croped,
        predicted_ts=pr_mask,
        pr_mask_onnx=pr_mask_onnx
    )

In [ ]:
type(ort_outs[0])

In [ ]:
with torch.no_grad():
#         %timeit logits = new_model(x_tensor)
        %timeit logits_std = model(x_tensor)
#         logits_croped = croped_model(x_tensor)

In [ ]:
values = np.random.randint(20, 100)
